In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectConfig
from loss_funcs import *
from algs import RandomRestarts, ParticleSwarm, SimulatedAnnealing, RandomSampling, DifferentialEvolution

from eval_funcs import *
from evaluator import Evaluator

# Create a camera configuration
cam_config = CameraConfig(position=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

loss_func = MSE()

alg1 = ParticleSwarm(sim_viewer, loss_func=loss_func)
alg1_config = ParticleSwarm.Config(time_limit=50, population=30, num_iters=300)

alg2 = SimulatedAnnealing(sim_viewer, loss_func=loss_func)
alg2_config = SimulatedAnnealing.Config(time_limit=50, num_iters=100, samples_per_temp=150)

alg3 = RandomSampling(sim_viewer, loss_func=loss_func)
alg3_config = RandomSampling.Config(time_limit=10, num_samples=10000)

alg4 = DifferentialEvolution(sim_viewer, loss_func=loss_func)
alg4_config = DifferentialEvolution.Config(time_limit=10, population=30, mut_prob=0.3)

evaluator = Evaluator(world_viewer, eval_func=IOU_Diff(method="mae"))

init_position = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
init_positions = [ObjectConfig(orient, init_position) for orient in random_orientations]

for alg, config in [(alg2, alg2_config)]:
    losses = evaluator.evaluate(alg, config, init_positions)
    print(f"{type(alg).__name__}: {losses}")

In [ ]:
""" import numpy as np

# Initialize the figure and axis
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
ax.set_xlim([-1, 2 * np.pi])
ax.set_ylim([-1, 2 * np.pi])
ax.set_zlim([-1, 2 * np.pi])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Initialize the plot with an empty scatter
scatter = ax.scatter([], [], [], c=[], cmap='viridis', s=5)

# Lists to store the points and matching values
points = []
matching_values = []


bm = BlitManager(fig.canvas, [scatter])
# make sure our window is on the screen and drawn
plt.show(block=False)
plt.pause(0.1)


def callback_function(orient, loss):
    global points, matching_values

    x, y, z = orient
    m = loss

    points.append((x, y, z))
    matching_values.append(m)
    
    # Update the scatter data with the new point
    scatter._offsets3d = np.array(points).T
    scatter._facecolors = plt.cm.viridis(np.array(matching_values) / max(matching_values))

    bm.update() """


from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectConfig
from loss_funcs import *
from algs.random_sampling import RandomSampling
from algs.particle_swarm import ParticleSwarm
from algs.simulated_annealing import SimulatedAnnealing
from algs.random_restarts import RandomRestarts
from algs.differential_evolution import DifferentialEvolution

from eval_funcs import *
from evaluator import Evaluator

# Create a camera configuration
cam_config = CameraConfig(position=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

loss_func = MSE()

alg1 = ParticleSwarm(sim_viewer, loss_func=loss_func)
alg1_config = ParticleSwarm.Config(time_limit=50, population=30, num_iters=300)

alg1.register_loss_callback(callback_function)

alg2 = SimulatedAnnealing(sim_viewer, loss_func=loss_func)
alg2_config = SimulatedAnnealing.Config(time_limit=10, num_iters=100, samples_per_temp=150)

alg3 = RandomSampling(sim_viewer, loss_func=loss_func)
alg3_config = RandomSampling.Config(time_limit=10, num_samples=10000)

alg4 = DifferentialEvolution(sim_viewer, loss_func=loss_func)
alg4_config = DifferentialEvolution.Config(time_limit=10, population=30, mut_prob=0.3)

evaluator = Evaluator(world_viewer, eval_func=IOU_Diff(method="mae"))

init_position = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
init_positions = [ObjectConfig(orient, init_position) for orient in random_orientations]

for alg, config in [(alg1, alg1_config)]:
    losses = evaluator.evaluate(alg, config, init_positions)
    print(f"{type(alg).__name__}: {losses}")

In [ ]:
cv.destroyAllWindows()